In [1]:
import pandas as pd
import copy

In [2]:
def footprint(activity) : 
    trace=list(set(activity))
    trace.sort()
    df=pd.DataFrame([['#']*len(activity) for a in activity],index = activity, columns=activity)
    return df

In [3]:
#depreciated by writeFootPrint2 (new version)
def writeFootPrint(df, trace) :
    for i in range(len(trace)-1) : 
        p=(trace[i:i+2])
        if df.loc[p[0],p[1]]=='#' :
            df.loc[p[0],p[1]]='>'
            df.loc[p[1],p[0]]='<'
        elif df.loc[p[0],p[1]] =='<' :
            df.loc[p[0],p[1]] = '||'
            df.loc[p[1],p[0]] = '||'
    return df

In [4]:
#the function for the single trace to create footprint 
def writeFootPrint2(direct_relation,parallel_relation, df, trace) :
    for i in range(len(trace)-1) : 
        p=(trace[i:i+2])
        #print(p)
        
        if df.loc[p[0],p[1]]=='#' :
            df.loc[p[0],p[1]]='>'
            df.loc[p[1],p[0]]='<'
            direct_relation.append(p)
        elif df.loc[p[0],p[1]] =='<' :
            df.loc[p[0],p[1]] = '||'
            df.loc[p[1],p[0]] = '||'
            try :
                direct_relation.remove(p)
#               print("remove",p)
            except :
                pass
#                 print("error")
                
            try :
                p.reverse()
                direct_relation.remove(p)
                #print("remove -r ",p)
                
            except :
                pass
                #print("error-r")    
               
            
            parallel_relation.append(p)
    return direct_relation, parallel_relation, df

In [5]:
#from the direct relationship, it creates merged relationship based on the footprint. 
def merge(candidate,complete) :
    print(footprint)
    candidate2=[]
    for i in range(len(candidate)-1) :
        j2=len(candidate)-1-i
        for j in range(j2) :
            a=candidate[j]
            b=candidate[j2]
            lhs=a[0]==b[0]
            rhs=a[1]==b[1]
#             print(j,j2, a, b, a[0]==b[0], a[1]==b[1])
            if lhs:
#                 print("lhs",a,b)
                p=list(
                    (set(a[1]+b[1]) - set(a[1])) | 
                    (set(a[1]+b[1]) - set(b[1]))
                    )
#                 print("lhs - p", p)
                if (complete.loc[p[0],p[1]]) == '#' :
                    m=list(set(a[1]) | set(b[1]))
                    m.sort()
                    item =  [a[0], (''.join(m))]
#                     print(item, "added lhs")
                    if item not in candidate2 : candidate2.append(item)
            if rhs:
#                 print("rhs",a,b)
                p=list(
                    (set(a[0]+b[0]) - set(a[0])) | 
                    (set(a[0]+b[0]) - set(b[0]))
                    )
#                 print("rhs - p", p)
                if (complete.loc[p[0],p[1]]) == '#' :
                    m=list(set(a[0]) | set(b[0]))
                    m.sort()
                    item =  [(''.join(m)), a[1]]
#                     print(item, "added rhs")
                    if item not in candidate2 : candidate2.append(item)
    candidate2.sort()
    return candidate2

In [6]:
#LOG Preprocessing. From the format of the lecture 
def logProcessing(L) :
    L=L.replace("<", "[").replace(">","]").replace(",","").replace(" ","").replace("\n","")[1:-1]
    T=[]
    for l in L : 
        if l=="[" : log=[]; continue
        if l=="]" : T.append(log); log=[]; continue
        log.append(l)
    return T

In [27]:
def createFootPrint(T) :
        
    logs=[]
    for l in T : 
        logs+=l

    activity=list(set(logs))
    activity.sort()
    df=footprint(activity)
    
    complete=pd.DataFrame([['#']*len(activity) for a in activity],index = activity, columns=activity) 

    
    parallel_relation=[]
    direct_relation=[]
    for trace in T : 
        direct_relation, parallel_relation,complete=writeFootPrint2(direct_relation, parallel_relation, df,trace)
    
    
    return activity, direct_relation, parallel_relation,complete

In [109]:
def alpha(L) : 
    T=logProcessing(L)

    activity, direct_relation, parallel_relation, complete = createFootPrint(T)
    direct_relation.sort()

    xl=copy.deepcopy(direct_relation)
    data=copy.deepcopy(direct_relation)
    while(True) :
        result=merge(data,complete)
        result.sort()
    #     print(result)
        if len(result)==0 : break
        xl += result 

        data=result

    yl=copy.deepcopy(xl)
    xl.sort(key=lambda e : len(str(e[0]))*3 + len(str(e[1])),reverse=True)
    for i in range(len(xl)) : 
        for j in range(i+1,len(xl)):
            l1=set(list("".join(xl[i][0])))
            l2=set(list("".join(xl[j][0])))
            r1=set(list("".join(xl[i][1])))
            r2=set(list("".join(xl[j][1])))
            
#             print(l1,l2,r1,r2)
            if (l1==l2 or r1==r2): 
                if (l2.issubset(l1) and l2!=l1) or (r2.issubset(r1) and r2!=r1) : 
                    print(xl[i],xl[j])
                    try : 
                        yl.remove(xl[j])
                    except :
                        pass
    yl.sort(key=lambda x : len(str(x)))

    return complete, xl, yl

## Comparision

In [124]:
# T = sets of sequences, in multi level arrays 
#Extract the elements from the logs
T=logProcessing(L_ass2)
activity, direct_relation, parallel_relation, complete = createFootPrint(T)
DF=[] 

for j in range(len(T)) : # 0 to 8
    #initialization for the foot print consisting of only "#"
    df=pd.DataFrame([['#']*len(activity) for a in activity],index = activity, columns=activity) 
    
    for idx, trace in enumerate(T) : 
        if j==idx : continue #artfically, omitting one trace for each loop 
        #print(j,idx)    
        df=writeFootPrint(df,trace)
    DF.append(df)

In [125]:
for i, df in enumerate(DF) :
    #print(complete==df)
    comp=(complete!=df)
    if sum(comp.sum(axis=1)) == 0 : 
        print(i,"same\n", df, )

2 same
    a  b  c   d   e   f   g   h  i  j
a  #  >  #   #   #   #   #   #  #  #
b  <  #  <   >   >   >   >   >  #  #
c  #  >  #   #   #   #   #   #  <  #
d  #  <  #   #  ||  ||  ||  ||  >  #
e  #  <  #  ||   #   #  ||  ||  >  #
f  #  <  #  ||   #   #  ||  ||  >  #
g  #  <  #  ||  ||  ||   #   #  >  #
h  #  <  #  ||  ||  ||   #   #  >  #
i  #  #  >   <   <   <   <   <  #  >
j  #  #  #   #   #   #   #   #  <  #


In [ ]:
complete

# Above do not touch


In [88]:
#assign 5-L1
L_ass1= '''
[
<a, d, f, h>, 
<a, c, e, g, c, e, h>, 
<a, d, f, g, d, f, h>, 
<a, f, g, d, f, h>
]
'''



In [89]:
#Assign 5-L2
L_ass2='''
[<a, b, d, e, g, i, j>, <a, b, e, d, h, i, j>, <a, b, f, d, g, i, j>, <a, b, f, g, d, i, j>, <a, b, g, d, f, i, j>,
<a, b, d, f, h, i, c, b, e, h, d, i, j>, <a, b, g, f, d, i, c, b, h, f, d, i, j>, <a, b, h, e, d, i, c, b, d, g, e, i, j>
]
'''

In [11]:
#Exercise 4-1
L1 = '''[< a, b, c >, < a, c >]''' #checked

In [12]:
#Exercise 4-2
L2=''' [<a, b, c, d, e, f, b, d, c, e, g>,
<a, b, d, c, e, g>,
<a, b, c, d, e, f, b, c, d, e, f, b, d, c, e, g>]'''#checked

In [13]:
#Exercise 4-3
L3 =''' 
[<a, b, e, f>,
<a, b, e, c, d, b, f>,
<a, b, c, e, d, b, f> , 
<a, b, c, d, e, b, f>,
<a, e, b, c, d, b, f> 
]''' #Checked

In [14]:
#Exercise 4-4
L4 ='''
[<a, d, e, f, h>,
<a, e, d, f, h>,
<g, h>,
<a, b, c, d, f, h>,
<a, c, b, d, f, h>,
<a, b, d, c, f, h>,
<a, c, d, b, f, h>]'''

In [15]:
#Exercise 4-5
L5 ='''
[<a, d, f, h>,
<a, c, e, g, c, e, h>,
<b, f, g, d, f, h>]'''

In [16]:
#Exercise 4-6-1
L6_1 = '''[<a, b, c>, <e, b, c>, <e, b, d>, <e, b, f>, <a, b, d>, <a, b, f>]'''

In [17]:
#Exercise 4-6-2
L6_2 = '''[<a, b, c>, <e, b, f>, <a, b, d>]'''

In [115]:
fp, xl, yl = alpha(L_ass2)

<function footprint at 0x7f870dce4d08>
<function footprint at 0x7f870dce4d08>
['ac', 'b'] ['a', 'b']
['ac', 'b'] ['c', 'b']
['ef', 'i'] ['e', 'i']
['ef', 'i'] ['f', 'i']
['gh', 'i'] ['g', 'i']
['gh', 'i'] ['h', 'i']
['b', 'ef'] ['b', 'e']
['b', 'ef'] ['b', 'f']
['b', 'gh'] ['b', 'g']
['b', 'gh'] ['b', 'h']
['i', 'cj'] ['i', 'c']
['i', 'cj'] ['i', 'j']


In [116]:
xl.sort(key=lambda x : len(str(x[0]))*9 + len(str(x[1])))

In [117]:
xl

[['a', 'b'],
 ['b', 'd'],
 ['b', 'e'],
 ['b', 'f'],
 ['b', 'g'],
 ['b', 'h'],
 ['c', 'b'],
 ['d', 'i'],
 ['e', 'i'],
 ['f', 'i'],
 ['g', 'i'],
 ['h', 'i'],
 ['i', 'c'],
 ['i', 'j'],
 ['b', 'ef'],
 ['b', 'gh'],
 ['i', 'cj'],
 ['ac', 'b'],
 ['ef', 'i'],
 ['gh', 'i']]

In [118]:
yl

[['b', 'd'],
 ['d', 'i'],
 ['ac', 'b'],
 ['b', 'ef'],
 ['b', 'gh'],
 ['ef', 'i'],
 ['gh', 'i'],
 ['i', 'cj']]

In [119]:
fp

,a,b,c,d,e,f,g,h,i,j
a,#,>,#,#,#,#,#,#,#,#
b,<,#,<,>,>,>,>,>,#,#
c,#,>,#,#,#,#,#,#,<,#
d,#,<,#,#,||,||,||,||,>,#
e,#,<,#,||,#,#,||,||,>,#
f,#,<,#,||,#,#,||,||,>,#
g,#,<,#,||,||,||,#,#,>,#
h,#,<,#,||,||,||,#,#,>,#
i,#,#,>,<,<,<,<,<,#,>
j,#,#,#,#,#,#,#,#,<,#


In [120]:
direct_relation

[['a', 'b'],
 ['b', 'd'],
 ['g', 'i'],
 ['i', 'j'],
 ['b', 'e'],
 ['h', 'i'],
 ['b', 'f'],
 ['d', 'i'],
 ['b', 'g'],
 ['f', 'i'],
 ['i', 'c'],
 ['c', 'b'],
 ['b', 'h'],
 ['e', 'i']]

In [121]:
parallel_relation

[['d', 'e'],
 ['d', 'g'],
 ['f', 'd'],
 ['d', 'h'],
 ['f', 'g'],
 ['f', 'h'],
 ['e', 'h'],
 ['e', 'g']]